In [0]:
# Caminho para a camada Gold
camada_gold_base = "/dbfs/FileStore/tables/data_lake/gold/"

# Ler as tabelas de dados transformados na camada Silver
df_clientes = spark.read.format("delta").load("/dbfs/FileStore/tables/data_lake/silver/clientes/")
df_produtos = spark.read.format("delta").load("/dbfs/FileStore/tables/data_lake/silver/produtos/")
df_pedidos = spark.read.format("delta").load("/dbfs/FileStore/tables/data_lake/silver/pedidos/")
df_pagamentos = spark.read.format("delta").load("/dbfs/FileStore/tables/data_lake/silver/pagamentos/")
df_entregas = spark.read.format("delta").load("/dbfs/FileStore/tables/data_lake/silver/entregas/")
df_categorias = spark.read.format("delta").load("/dbfs/FileStore/tables/data_lake/silver/categorias/")

# =====================================
# Fatos
# =====================================

# Fato de Pedidos: Agregar dados de pedidos com informações do cliente e valor total
df_fato_pedidos = df_pedidos.join(df_clientes, df_pedidos.id_cliente == df_clientes.id_cliente) \
    .select("id_pedido", "id_cliente", "data_pedido", "valor_total", "nome", "email")

# Salvar a tabela de Fato de Pedidos na camada Gold
df_fato_pedidos.write.format("delta").mode("overwrite").save(f"{camada_gold_base}fatos/fato_pedidos")

# Fato de Pagamentos: Agregar dados de pagamentos com informações de pedidos
df_fato_pagamentos = df_pagamentos.join(df_pedidos, df_pagamentos.id_pedido == df_pedidos.id_pedido) \
    .select("id_pagamento", "id_pedido", "valor_pago", "data_pagamento", "metodo_pagamento")

# Salvar a tabela de Fato de Pagamentos na camada Gold
df_fato_pagamentos.write.format("delta").mode("overwrite").save(f"{camada_gold_base}fatos/fato_pagamentos")

# Fato de Entregas: Agregar dados de entregas com informações de pedidos
df_fato_entregas = df_entregas.join(df_pedidos, df_entregas.id_pedido == df_pedidos.id_pedido) \
    .select("id_entrega", "id_pedido", "data_entrega", "status_entrega", "endereco_entrega")

# Salvar a tabela de Fato de Entregas na camada Gold
df_fato_entregas.write.format("delta").mode("overwrite").save(f"{camada_gold_base}fatos/fato_entregas")

# =====================================
# Dimensões
# =====================================

# Dimensão de Produtos: Detalhamento de produtos com informações de categoria
df_dim_produtos = df_produtos.join(df_categorias, df_produtos.id_categoria == df_categorias.id_categoria) \
    .select("id_produto", "nome", "preco", "estoque", "id_categoria", "nome_categoria", "descricao")

# Salvar a tabela de Dimensão de Produtos na camada Gold
df_dim_produtos.write.format("delta").mode("overwrite").save(f"{camada_gold_base}dimensoes/dim_produtos")

# Dimensão de Clientes: Detalhamento de clientes
df_dim_clientes = df_clientes.select("id_cliente", "nome", "email", "data_criacao")

# Salvar a tabela de Dimensão de Clientes na camada Gold
df_dim_clientes.write.format("delta").mode("overwrite").save(f"{camada_gold_base}dimensoes/dim_clientes")

# Dimensão de Categorias: Detalhamento de categorias
df_dim_categorias = df_categorias.select("id_categoria", "nome", "descricao")

# Salvar a tabela de Dimensão de Categorias na camada Gold
df_dim_categorias.write.format("delta").mode("overwrite").save(f"{camada_gold_base}dimensoes/dim_categorias")

print("Transformações para Fatos e Dimensões na camada Gold concluídas.")
